### 3-1-ResizePredictions.ipynb 

Rezises segmentations using the format of the original image.

In [1]:
import os
import cv2
import numpy as np
import rasterio.features
from shapely import Polygon
from pathlib import Path
from tqdm import tqdm

In [3]:
# Define your base working directory
base_dir = Path("/mnt/c/Projects/Master/Data/Processed/LiveBees/8-SemanticSegmentation/")
    
# Build directory paths from the base directory
pred_fps = list((base_dir / "predictions").glob("*"))
wing_fp = base_dir / "to_predict" / "images"
out_fp = base_dir / "predictions_resized"

In [5]:
DEBUG = False

# Loop through every prediction
for pred_fp in tqdm(pred_fps, desc="Processing files", ncols=145):
    # Determine file paths
    pred_fp = Path(pred_fp)
    pred_fn = pred_fp.name
    seg_fp = Path(wing_fp) / pred_fn
    os.makedirs(out_fp, exist_ok=True)
    new_fp = Path(out_fp) / pred_fn
    
    # Skip if output file exists
    if os.path.exists(new_fp):
        continue   
            
    # Through an error if no matching image exists
    if not os.path.exists(seg_fp):
        print("Warning: Missing Image!")
        continue

    # Open the matching image pair
    seg = cv2.imread(seg_fp)
    pred = cv2.imread(pred_fp, cv2.IMREAD_GRAYSCALE)

    # Calculate the image scale
    x_scale = seg.shape[1] / pred.shape[1]
    y_scale = seg.shape[0] / pred.shape[0]

    # Resize the prediction using the scale
    out = np.zeros(seg.shape[:2]).astype("uint8")
    for class_id in range(1, 7):
        if (pred == class_id).sum() > 0:
    
            contours, hierarchy = cv2.findContours((pred == class_id).astype("uint8"), 1, 2)
            area_sorted_indices = np.argsort([cv2.contourArea(x) for x in contours])
            biggest_contour_index = area_sorted_indices[-1]
            biggest_contour = contours[biggest_contour_index].squeeze()
    
            # biggest_contour = get_biggest_contour()
            if len(biggest_contour) >= 4:
                poly = Polygon(biggest_contour*np.array([x_scale, y_scale]))
                class_mask_resized = rasterio.features.rasterize([poly], out_shape=(int(256 * y_scale), int(256 * x_scale)))
                out[np.where(class_mask_resized > 0.5)] = class_id
    if DEBUG:
        plt.imshow(out.astype("uint8"))

    # Save the rescaled prediction
    cv2.imwrite(new_fp, out)

Processing files: 100%|█████████████████████████████████████████████████████████████████████████████████████| 5574/5574 [00:39<00:00, 141.79it/s]
